In [168]:
import yfinance as yf
import dash
from dash import html, dcc, dash_table, callback_context
import plotly.graph_objects as go
# import dash_trich_components as dtc
from dash.dependencies import Input, Output
# import dash_bootstrap_components as dbc
import plotly.express as px
import json
import pandas as pd
pd.set_option("display.max_rows", None)
# from pandas_ta import bbands
# import pandas_datareader as web
import numpy as np
import datetime
from scipy.stats import pearsonr
import sqlite3
import requests
from bs4 import BeautifulSoup
import re
import datetime
import pytz


In [169]:
r = requests.get('https://www.marketbeat.com/types-of-stock/sp-500-stocks/')


In [170]:
soup = BeautifulSoup(r.text, 'lxml')


In [171]:
results = soup.find_all('div', class_='ticker-area')


In [172]:
# tickers ordered by Market Capital
ticker_symbols = [res.text for res in results]
ticker_symbols[:10]
    

['AAPL',
 'MSFT',
 'GOOG',
 'GOOGL',
 'AMZN',
 'NVDA',
 'TSLA',
 'META',
 'BRK.B',
 'UNH']

In [173]:
con = sqlite3.connect('stonks.db')
cur = con.cursor()


In [174]:
def create_tables(drop=False):
    create_tables = '''
    CREATE TABLE IF NOT EXISTS exchange (
    id INTEGER PRIMARY KEY NOT NULL,
    name TEXT NOT NULL UNIQUE);
    
    CREATE TABLE IF NOT EXISTS ticker_type (
    id INTEGER PRIMARY KEY NOT NULL,
    name TEXT NOT NULL UNIQUE);
    
    CREATE TABLE IF NOT EXISTS sector (
    id INTEGER PRIMARY KEY NOT NULL,
    name TEXT NOT NULL UNIQUE);
    
    CREATE TABLE IF NOT EXISTS currency (
    id INTEGER PRIMARY KEY NOT NULL,
    iso_code TEXT NOT NULL UNIQUE);
    
    CREATE TABLE IF NOT EXISTS ticker (
    id INTEGER PRIMARY KEY NOT NULL,
    currency_id INTEGER NOT NULL,
    exchange_id INTEGER NOT NULL,
    ticker_type_id INTEGER NOT NULL,
    sector_id INTEGER NOT NULL,
    symbol TEXT NOT NULL UNIQUE,
    name TEXT NOT NULL,
    FOREIGN KEY(currency_id) REFERENCES currency(id),
    FOREIGN KEY(exchange_id) REFERENCES exchange(id),
    FOREIGN KEY(ticker_type_id) REFERENCES ticker_type(id),
    FOREIGN KEY(sector_id) REFERENCES sector(id));
    
    CREATE TABLE IF NOT EXISTS date (
    id INTEGER PRIMARY KEY NOT NULL,
    date INTEGER NOT NULL UNIQUE);
    
    CREATE TABLE IF NOT EXISTS price (
    ticker_id INTEGER NOT NULL,
    date_id INTEGER NOT NULL,
    open REAL NOT NULL,
    high REAL NOT NULL,
    low REAL NOT NULL,
    close REAL NOT NULL,
    volume INTEGER NOT NULL,
    PRIMARY KEY (ticker_id,date_id),
    FOREIGN KEY(ticker_id) REFERENCES ticker(id),
    FOREIGN KEY(date_id) REFERENCES date(id));
    
    CREATE INDEX IF NOT EXISTS fk_ticker_idx
    on ticker (sector_id, exchange_id, currency_id, ticker_type_id);
    '''
    drop_tables = '''
    DROP TABLE IF EXISTS ticker;
    DROP TABLE IF EXISTS exchange;
    DROP TABLE IF EXISTS price;
    DROP TABLE IF EXISTS date;
    DROP TABLE IF EXISTS ticker_type;
    DROP TABLE IF EXISTS sector;
    DROP TABLE IF EXISTS currency;
    '''
    if drop:
        cur.executescript(drop_tables)
    cur.executescript(create_tables)
    con.commit()

create_tables(True)
    

In [175]:
def list_indexes():
        # Get all tables from sqlite_master
        cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
        tables = cur.fetchall()
        for table in tables:
            table_name = table[0]
            # Get table info
            cur.execute(f"PRAGMA table_info({table_name});")
            columns = cur.fetchall()
            # Get index info
            cur.execute(f"PRAGMA index_list({table_name});")
            indexes = cur.fetchall()
            print('Table:', table_name)
            
            # Extract and display info about each individual index
            for index in indexes:
                index_name = index[1]
                unique = index[2]
                index_columns = []
                
                for column in columns:
                    cur.execute(f"PRAGMA index_info({index_name});")
                    index_info = cur.fetchall()
                    for info in index_info:
                        if info[2] == column[1]:
                            index_columns.append(column[1])
                print('  Index:', index_name)
                print('    Columns:', ', '.join(index_columns))
                print('    Unique:', 'Yes' if unique else 'No')
            print()
            
list_indexes()


Table: exchange
  Index: sqlite_autoindex_exchange_1
    Columns: name
    Unique: Yes

Table: ticker_type
  Index: sqlite_autoindex_ticker_type_1
    Columns: name
    Unique: Yes

Table: sector
  Index: sqlite_autoindex_sector_1
    Columns: name
    Unique: Yes

Table: currency
  Index: sqlite_autoindex_currency_1
    Columns: iso_code
    Unique: Yes

Table: ticker
  Index: fk_ticker_idx
    Columns: currency_id, exchange_id, ticker_type_id, sector_id
    Unique: No
  Index: sqlite_autoindex_ticker_1
    Columns: symbol
    Unique: Yes

Table: date
  Index: sqlite_autoindex_date_1
    Columns: date
    Unique: Yes

Table: price
  Index: sqlite_autoindex_price_1
    Columns: ticker_id, date_id
    Unique: Yes



In [176]:
ticker = yf.Ticker('AAPL')
info = ticker.info
cols = [
        'symbol',
        'shortName',
        'sector',
        'exchange',
        'quoteType',
        'timeZoneShortName',
        'gmtOffSetMilliseconds',
        'currency',
       ]

pd.DataFrame(ticker.info).iloc[0][cols].to_frame()


,0
symbol,AAPL
shortName,Apple Inc.
sector,Technology
exchange,NMS
quoteType,EQUITY
timeZoneShortName,EDT
gmtOffSetMilliseconds,-14400000
currency,USD


In [177]:
# Initiate tickers instance 
tickers = yf.Tickers(' '.join(ticker_symbols[:30]))


In [178]:
temp = tickers.download(group_by='ticker', start='2023-07-06')
temp.drop(columns=['Adj Close', 'Dividends', 'Stock Splits'], level=1, inplace=True)
temp.index = temp.index.astype('datetime64[s]').astype(int)
[(unix,) for unix in temp.index]


[*********************100%***********************]  30 of 30 completed


1 Failed download:
['BRK.B']: Exception('%ticker%: No timezone found, symbol may be delisted')


[(1688601600,),
 (1688688000,),
 (1688947200,),
 (1689033600,),
 (1689120000,),
 (1689206400,),
 (1689292800,),
 (1689552000,),
 (1689638400,),
 (1689724800,),
 (1689811200,),
 (1689897600,)]

In [179]:
pd.to_datetime(temp.index, unit='s')


DatetimeIndex(['2023-07-06', '2023-07-07', '2023-07-10', '2023-07-11',
               '2023-07-12', '2023-07-13', '2023-07-14', '2023-07-17',
               '2023-07-18', '2023-07-19', '2023-07-20', '2023-07-21'],
              dtype='datetime64[ns]', name='Date', freq=None)

In [180]:
# Fill database with ticker information
for symbol in ticker_symbols[:30]:
    try:
        with con:          
            info = tickers.tickers[symbol].info
            con.execute('''
            INSERT OR IGNORE INTO currency (iso_code)
            VALUES (:currency)
            ''', info)
            con.execute('''
            INSERT OR IGNORE INTO exchange (name)
            VALUES (:exchange)
            ''', info)
            con.execute('''
            INSERT OR IGNORE INTO ticker_type (name)
            VALUES (:quoteType)
            ''', info)
            con.execute('''
            INSERT OR IGNORE INTO sector (name)
            VALUES (:sector)
            ''', info)
            con.execute('''
            INSERT INTO ticker (name, symbol,
            currency_id, exchange_id, ticker_type_id, sector_id
            )
            VALUES (:shortName, :symbol,
            (SELECT id FROM currency WHERE iso_code=:currency), 
            (SELECT id FROM exchange WHERE name=:exchange),
            (SELECT id FROM ticker_type WHERE name=:quoteType),
            (SELECT id FROM sector WHERE name=:sector)
            )
            ''', info)
            print(symbol)
    except:
        print(f"failed to insert {symbol}")
    


AAPL
MSFT
GOOG
GOOGL
AMZN
NVDA
TSLA
META
failed to insert BRK.B
UNH
JPM
V
JNJ
LLY
WMT
XOM
MA
AVGO
PG
HD
ORCL
CVX
MRK
KO
PEP
BAC
ABBV
COST
ADBE
CRM


In [181]:
%%time
# Batch retrieve OHLC data (≈x3 times faster, but higher CPU time)
data = tickers.download(start='2022-07-01', end='2023-07-01', group_by='ticker')
data.drop(columns=['Adj Close', 'Dividends', 'Stock Splits'], level=1, inplace=True)
# Convert the date to a unix timestamp (seconds)
data.index = data.index.astype('datetime64[s]').astype(int)
# Inserting date
con.executemany('''
INSERT OR IGNORE INTO date (date)
VALUES (?)
''', [(unix,) for unix in data.index])
for symbol in cur.execute('select symbol from ticker').fetchall():
    try:
        # Retrieve OHLC data for symbol
        ohlc_data = data[symbol[0]].reset_index()
        # Convert to a list of dictionaries (records)
        ohlc_data = ohlc_data.to_dict(orient='records')
        with con:          
            # Using an f-string is an SQL injection vulnerability,
            # but given the context it doesn't matter
            con.executemany(f'''
            INSERT INTO price (ticker_id, date_id,
            open, high, low, close, volume
            )
            VALUES (
            (SELECT id FROM ticker WHERE symbol='{symbol[0]}'),
            (SELECT id FROM date WHERE date=:Date),
            :Open, :High, :Low, :Close, :Volume
            )
            ''', ohlc_data)
            print(symbol[0])
    except Exception as e:
        print(f"[{symbol[0]}] {e}")
        

[*********************100%***********************]  30 of 30 completed


1 Failed download:
['BRK.B']: Exception('%ticker%: No timezone found, symbol may be delisted')



AAPL
ABBV
ADBE
AMZN
AVGO
BAC
COST
CRM
CVX
GOOG
GOOGL
HD
JNJ
JPM
KO
LLY
MA
META
MRK
MSFT
NVDA
ORCL
PEP
PG
TSLA
UNH
V
WMT
XOM
CPU times: user 7.15 s, sys: 2.08 s, total: 9.23 s
Wall time: 9.79 s


In [182]:
%%script echo skipping ᘇᘏᗢ、
%%time
# Per ticker OHLC data retrieval (lower CPU time, but ≈x3 times slower) - helps avoid rate limiting
for symbol in cur.execute('select symbol from ticker').fetchall():
    try:
        # Retrieve OHLC data for symbol
        ohlc_data = tickers.tickers[symbol[0]].history(start='2022-07-01', end='2023-07-01')[['Open', 'High', 'Low', 'Close', 'Volume']]
        # Convert the date to a unix timestamp (remove timezone holding local time representations)
        ohlc_data.index = ohlc_data.index.tz_localize(None).astype('int')/10**9
        ohlc_data.reset_index(inplace=True)
        # Convert to a list of dictionaries (records)
        ohlc_data = ohlc_data.to_dict(orient='records')
        with con:          
            # Inserting date could be optimized
            con.executemany('''
            INSERT OR IGNORE INTO date (date)
            VALUES (:Date)
            ''', ohlc_data)
            
            # Using an f-string is an SQL injection vulnerability,
            # but given the context it doesn't matter
            con.executemany(f'''
            INSERT INTO price (ticker_id, date_id,
            open, high, low, close, volume
            )
            VALUES (
            (SELECT id FROM ticker WHERE symbol='{symbol[0]}'),
            (SELECT id FROM date WHERE date=:Date),
            :Open, :High, :Low, :Close, :Volume
            )
            ''', ohlc_data)
            print(symbol[0])
    except Exception as e:
        print(f"[{symbol[0]}] {e}")


skipping ᘇᘏᗢ、


In [183]:
cur.execute('''
select DATE(max(date)+86400, 'unixepoch')
from price p join ticker t
    on t.id=p.ticker_id
join date d
    on p.date_id=d.id
where t.symbol='AAPL'
''').fetchone()[0]


'2023-07-01'

In [184]:
def add_new_ohlc(symbol):
    try:
        # Get date for latest entry
        latest_entry = cur.execute('''
        select DATE(max(date)+86400, 'unixepoch')
        from price p join ticker t
            on t.id=p.ticker_id
        join date d
            on p.date_id=d.id
        where t.symbol=?
        ''', (symbol,)).fetchone()[0]
        # Retrieve new OHLC data for symbol
        ohlc_data = tickers.tickers[symbol].history(start=latest_entry)[['Open', 'High', 'Low', 'Close', 'Volume']]
        # Convert the date to a unix timestamp (remove timezone holding local time representations)
        ohlc_data.index = ohlc_data.index.tz_localize(None).astype('int')/10**9
        ohlc_data.reset_index(inplace=True)
        # Convert to a list of dictionaries (records)
        ohlc_data = ohlc_data.to_dict(orient='records')
        with con:          
            # Inserting date could be optimized
            con.executemany('''
            INSERT OR IGNORE INTO date (date)
            VALUES (:Date)
            ''', ohlc_data)
            
            # Using an f-string is an SQL injection vulnerability,
            # but given the context it doesn't matter
            con.executemany(f'''
            INSERT INTO price (ticker_id, date_id,
            open, high, low, close, volume
            )
            VALUES (
            (SELECT id FROM ticker WHERE symbol='{symbol}'),
            (SELECT id FROM date WHERE date=:Date),
            :Open, :High, :Low, :Close, :Volume
            )
            ''', ohlc_data)
            print(symbol)
    except Exception as e:
        print(f"[{symbol}] {e}")

add_new_ohlc('AAPL')
                                    

AAPL


In [185]:
# Check number of entries per ticket name (not symbol)
cur.execute('select count(), name from price join ticker on ticker.id=price.ticker_id group by ticker.name').fetchall()


[(251, 'AbbVie Inc.'),
 (251, 'Adobe Inc.'),
 (502, 'Alphabet Inc.'),
 (251, 'Amazon.com, Inc.'),
 (265, 'Apple Inc.'),
 (251, 'Bank of America Corporation'),
 (251, 'Broadcom Inc.'),
 (251, 'Chevron Corporation'),
 (251, 'Coca-Cola Company (The)'),
 (251, 'Costco Wholesale Corporation'),
 (251, 'Eli Lilly and Company'),
 (251, 'Exxon Mobil Corporation'),
 (251, 'Home Depot, Inc. (The)'),
 (251, 'JP Morgan Chase & Co.'),
 (251, 'Johnson & Johnson'),
 (251, 'Mastercard Incorporated'),
 (251, 'Merck & Company, Inc.'),
 (251, 'Meta Platforms, Inc.'),
 (251, 'Microsoft Corporation'),
 (251, 'NVIDIA Corporation'),
 (251, 'Oracle Corporation'),
 (251, 'Pepsico, Inc.'),
 (251, 'Procter & Gamble Company (The)'),
 (251, 'Salesforce, Inc.'),
 (251, 'Tesla, Inc.'),
 (251, 'UnitedHealth Group Incorporated'),
 (251, 'Visa Inc.'),
 (251, 'Walmart Inc.')]

In [186]:
# List all dates
cur.execute("select DATE(date, 'unixepoch') from date").fetchall()


[('2022-07-01',),
 ('2022-07-05',),
 ('2022-07-06',),
 ('2022-07-07',),
 ('2022-07-08',),
 ('2022-07-11',),
 ('2022-07-12',),
 ('2022-07-13',),
 ('2022-07-14',),
 ('2022-07-15',),
 ('2022-07-18',),
 ('2022-07-19',),
 ('2022-07-20',),
 ('2022-07-21',),
 ('2022-07-22',),
 ('2022-07-25',),
 ('2022-07-26',),
 ('2022-07-27',),
 ('2022-07-28',),
 ('2022-07-29',),
 ('2022-08-01',),
 ('2022-08-02',),
 ('2022-08-03',),
 ('2022-08-04',),
 ('2022-08-05',),
 ('2022-08-08',),
 ('2022-08-09',),
 ('2022-08-10',),
 ('2022-08-11',),
 ('2022-08-12',),
 ('2022-08-15',),
 ('2022-08-16',),
 ('2022-08-17',),
 ('2022-08-18',),
 ('2022-08-19',),
 ('2022-08-22',),
 ('2022-08-23',),
 ('2022-08-24',),
 ('2022-08-25',),
 ('2022-08-26',),
 ('2022-08-29',),
 ('2022-08-30',),
 ('2022-08-31',),
 ('2022-09-01',),
 ('2022-09-02',),
 ('2022-09-06',),
 ('2022-09-07',),
 ('2022-09-08',),
 ('2022-09-09',),
 ('2022-09-12',),
 ('2022-09-13',),
 ('2022-09-14',),
 ('2022-09-15',),
 ('2022-09-16',),
 ('2022-09-19',),
 ('2022-09

In [187]:
main_query = '''
select t.symbol, t.name, s.name, DATE(d.date, 'unixepoch'),
    p.open, p.high, p.low, p.close, p.volume,
    e.name, tt.name, c.iso_code
from price p join ticker t
    on p.ticker_id=t.id
join date d
    on p.date_id=d.id
join sector s
    on t.sector_id=s.id
join exchange e
    on t.exchange_id=e.id
join currency c
    on t.currency_id=c.id
join ticker_type tt
    on t.ticker_type_id=tt.id
'''


In [188]:
cur.execute(main_query).fetchmany(3)


[('AAPL',
  'Apple Inc.',
  'Technology',
  '2022-07-01',
  135.23248650219884,
  138.21467908914858,
  134.85475242223865,
  138.10533142089844,
  71051600,
  'NMS',
  'EQUITY',
  'USD'),
 ('AAPL',
  'Apple Inc.',
  'Technology',
  '2022-07-05',
  136.95222989609488,
  140.76943280643897,
  136.11720443528156,
  140.7197265625,
  73353800,
  'NMS',
  'EQUITY',
  'USD'),
 ('AAPL',
  'Apple Inc.',
  'Technology',
  '2022-07-06',
  140.51099744017776,
  143.2645446803833,
  140.24259582717173,
  142.07167053222656,
  74064300,
  'NMS',
  'EQUITY',
  'USD')]

In [189]:
con.commit()
con.close()